<a href="https://colab.research.google.com/github/Maxim2k7/git_project1/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22tasks_RU_gc_rc_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задача 1:** Изучите как работает базовый reference counting

Создайте строку и исследуйте ее reference count на разных этапах. Верните кортеж: (count_after_creation, count_after_ref, count_after_del)

In [1]:
import sys
import gc
import weakref

In [2]:
def task1():
    s = str(12345)
    count_after_creation = sys.getrefcount(s)
    t = s
    count_after_ref = sys.getrefcount(s)
    del t
    count_after_del = sys.getrefcount(s)
    return (count_after_creation, count_after_ref, count_after_del)
    pass

# Проверка
print("Задача 1:", task1())

Задача 1: (2, 3, 2)


**Задача 2:** Влияние функций на Reference Counting

Создайте список и передайте его в функцию.
Сравните reference count до и после вызова функции.
Верните кортеж: (count_before_call, count_during_call, count_after_call)

In [3]:
def task2():

    def process_list(lst):
        return sys.getrefcount(lst)

    lst = [1, 2, 3]
    count_before_call = sys.getrefcount(lst)
    count_during_call = process_list(lst)
    count_after_call = sys.getrefcount(lst)
    return (count_before_call, count_during_call, count_after_call)
    pass

# Проверка
print("Задача 2:", task2())

Задача 2: (2, 3, 2)


**Задача 3:** Циклические ссылки и Memory Leaks

Создайте два объекта с циклической ссылкой, затем разорвите ее.
Используйте gc для проверки количества собранных объектов.
Верните количество объектов собранных gc после разрыва ссылки.

In [4]:
def task3():
    class Node:
        def __init__(self, name):
            self.name = name
            self.ref = None

    n1 = Node("Alice")
    n2 = Node("Bob")
    n1.ref = n2
    n2.ref = n1
    cur1 = gc.get_count()
    n1.ref = None
    del n1
    del n2
    cur2 = gc.get_count()
    return cur1[0] - cur2[0] + cur1[1] - cur2[1] + cur1[2] - cur2[2]
    pass

print("Задача 3:", task3())

Задача 3: 2


**Задача 4:** Сравнение Reference Count для разных типов данных

Сравните reference count для чисел, строк, списков и словарей.
Верните словарь с reference count для каждого типа после создания.

In [5]:
def task4():
    # Подсказка: можно посмотреть int, str, массив и другие
    res = {"int": sys.getrefcount(2355), "string": sys.getrefcount(str(123)), "list": sys.getrefcount([1, 2, 3]), "tuple": sys.getrefcount((2, 3, 9)), "dict": sys.getrefcount({0: 1, 2: 6})}
    return res
    pass

print("Задача 4:", task4())

Задача 4: {'int': 3, 'string': 1, 'list': 1, 'tuple': 2, 'dict': 1}


**Задача 5:** Weak References

Создайте два объекта со слабой ссылкой (weakref) между ними.
Убедитесь, что объекты могут быть удалены сборщиком мусора.
Верните True если weakref не увеличивает reference count.

In [6]:
def task5():
    class Data:
        def __init__(self, value):
            self.value = value

    A = Data("x");
    cnt1 = sys.getrefcount(A)
    B = Data(weakref.ref(A));
    cnt2 = sys.getrefcount(A)
    cur1 = sum(gc.get_count())
    del A
    del B
    cur2 = sum(gc.get_count())
    assert cur1 != cur2
    return cnt2 == cnt1
    pass

print("Задача 5:", task5())

Задача 5: True


**Задача 6:** Мониторинг работы Garbage Collector

Зарегистрируйте callback функцию для отслеживания работы GC.
Верните список событий, которые отследил callback.

In [7]:
def task6():
    events = []
    def callback(phase, info):
      events.append([phase, info])
    gc.callbacks.clear()
    gc.callbacks.append(callback)
    f = []
    del f
    gc.collect()
    return events
    pass

print("Задача 6:", task6())

Задача 6: [['start', {'generation': 2, 'collected': 0, 'uncollectable': 0}], ['stop', {'generation': 2, 'collected': 34, 'uncollectable': 0}]]


**Задача 7:** Анализ поколений в GC

Создайте объекты и отследите их перемещение между поколениями GC.
Верните кортеж с количеством объектов в каждом поколении до и после создания объектов.

In [8]:
def task7():
    f = []
    class Data:
      def __init__(self, num):
        self.ind = num
    cur1 = gc.get_count()
    for i in range(10000):
      f.append(Data(i))
      #cur2 = gc.get_count()
      #print(cur1, cur2)
    cur2 = gc.get_count()
    #print(cur1, cur2)
    #print(len(f))
    return (cur1, cur2)
    pass

print("Задача 7:", task7())

Задача 7: ((417, 0, 0), (590, 2, 1))


**Задача 8:** Мониторинг порогов сборки мусора

Изучите как меняются счетчики поколений GC при создании объектов.
Верните словарь с состоянием счетчиков до и после создания объектов.

In [10]:
def task8():
    initial_threshold = gc.get_threshold()
    initial_count = gc.get_count()
    class Data:
      pass
    A = Data()
    B = Data()
    C = Data()
    gc.collect()
    after_creation_count = gc.get_count()
    del A
    del B
    del C
    gc.collect()
    after_collect_count = gc.get_count()
    collected = sum(after_creation_count) - sum(after_collect_count)
    return {
        'thresholds': initial_threshold,
        'initial_count': initial_count,
        'after_creation': after_creation_count,
        'after_collect_gen0': after_collect_count,
        'collected_objects': collected
    }
    pass

print("Задача 8:", task8())

Задача 8: {'thresholds': (700, 10, 10), 'initial_count': (144, 1, 0), 'after_creation': (20, 0, 0), 'after_collect_gen0': (0, 0, 0), 'collected_objects': 20}


**Задача 9:** Быстрая проверка работы GC

Проверьте, собирает ли GC циклические ссылки.
Верните разницу в количестве объектов до и после сборки.

In [10]:
def task9():
    class Data:
      pass
    A = Data()
    B = Data()
    B.ref = A
    A.ref = B
    gc.collect()
    del A
    del B
    cur1 = gc.get_count()
    gc.collect()
    cur2 = gc.get_count()
    return sum(cur1) != sum(cur2)
    pass

print("Задача 9:", task9())

Задача 9: True


In [11]:
def task9_solution():
    # Создаем циклические ссылки быстро
    a, b, c = [], [], []
    a.append(b); b.append(c); c.append(a)  # Цикл a->b->c->a

    count_before = len(gc.get_objects())
    collected = gc.collect()
    count_after = len(gc.get_objects())

    return count_before - count_after, collected

print("Задача 9:", task9_solution())

Задача 9: (11, 5)


**Задача 10:** Детектор "неубиваемых" объектов

Найдите объекты, которые переживают принудительную сборку мусора.
Выведите количество таких объектов.

In [ ]:
def task10():
    class Cool:
        def __init__(self, ref=None):
            self.ref = ref
        def __del__(self):
            Cool(self)
    c = Cool()
    w = weakref.ref(c)
    gc.collect()
    del c
    cnt1 = gc.get_count()
    gc.collect()
    cnt2 = gc.get_count()
    gc.collect()
    cnt3 = gc.get_count()
    print(cnt1, cnt2, cnt3, w)
    pass

print("Задача 10:", task10())

Exception ignored in: <function task10.<locals>.Cool.__del__ at 0x787a33c42340>
Traceback (most recent call last):
  File "/tmp/ipython-input-1892870193.py", line 6, in __del__
KeyboardInterrupt: 
Exception ignored in: <function task6.<locals>.callback at 0x787a33c41da0>
Traceback (most recent call last):
  File "/tmp/ipython-input-636294890.py", line 3, in callback
KeyboardInterrupt: 


In [ ]:
# Небольшая подсказка для создания неудаляемых объектов
class Persistent:
        def __del__(self):
            pass  # Мешает нормальному удалению